In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
from torchvision.transforms import functional as F

In [2]:
#padding to square 
def pad_to_square(image):
    width, height = image.size
    max_wh = max(width, height)
    padding = [
        (max_wh - width) // 2,      # Left
        (max_wh - height) // 2,     # Top
        (max_wh - width + 1) // 2,  # Right
        (max_wh - height + 1) // 2  # Bottom
    ]
    return F.pad(image, padding, fill=0)
def bounding_box(image):
    gray_image = image.convert("L")
    binary_image = gray_image.point(lambda p: p > 0 and 255)
    bbox = binary_image.getbbox()
    return bbox
def crop_to_square(image):
    bbox = bounding_box(image)
    if bbox:
        left,top,right,bottom = bbox
        width, height = right - left, bottom - top
        max_size = max(width, height)
        new_left = left - (max_size - width) // 2
        new_top = top - (max_size - height) // 2
        new_right = new_left + max_size
        new_bottom = new_top + max_size
        return image.crop((new_left, new_top, new_right, new_bottom))
    return image

In [3]:
transform = transforms.Compose([
    transforms.Lambda(lambda img: pad_to_square(img)),
    transforms.Lambda(lambda img: crop_to_square(img)),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),  # Đảm bảo có dấu ngoặc đơn
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
dataset = datasets.ImageFolder(root='./Datasets', transform=transform)

In [5]:
print(dataset.class_to_idx)

{'no': 0, 'yes': 1}


In [6]:
train_size = int(0.6*len(dataset))
test_size = int(len(dataset)*0.2)
val_size = len(dataset) - test_size - train_size

In [7]:
from torch.utils.data import random_split
data_train, data_test, data_val = random_split(dataset, [train_size,test_size,val_size])

In [8]:
train_loader = DataLoader(data_train, batch_size=32, shuffle=True)
test_loader =  DataLoader(data_test, batch_size=32, shuffle=False)
val_loader =  DataLoader(data_val, batch_size=32, shuffle=False)


In [9]:
import torch.nn as nn
import torch.optim  as optim
import torch.nn.functional as f
# tính kích thước đầu ra:(Input Size−Kernel Size+2*padding)/S + 1
# (inputsize- kernel size)/s+1
#xây dựng mô hình CNN
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # Conv2D layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # MaxPooling layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        def get_output_size(input_size):
            size = input_size
            size = (size - 3 + 2 * 1) // 1 + 1  # Conv layer
            size = (size - 2) // 2 + 1  # MaxPooling layer
            size = (size - 3 + 2 * 1) // 1 + 1  # Conv layer
            size = (size - 2) // 2 + 1  # MaxPooling layer
            size = (size - 3 + 2 * 1) // 1 + 1  # Conv layer
            size = (size - 2) // 2 + 1  # MaxPooling layer
            return size

        input_size = 64
        output_size = get_output_size(input_size)
        self.fc1 = nn.Linear(64 * output_size * output_size, 64)
        self.fc2 = nn.Linear(64, 1)
        
        # Dropout
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(f.relu(self.conv1(x)))
        x = self.pool(f.relu(self.conv2(x)))
        x = self.pool(f.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = f.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)  # Remove sigmoid here
        return x
model = CNNModel()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
def calculate_accuracy(predictions, labels):
    predictions = torch.sigmoid(predictions)  # Convert logits to probabilities
    predicted_classes = (predictions > 0.5).float()  # Convert probabilities to class labels
    correct = (predicted_classes == labels).sum().item()
    return correct

In [11]:
train_losses = []
val_losses = []
val_accuracies = []
train_accuracies =[]

In [12]:
for epoch in range(20):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_train = 0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        optimizer.zero_grad()  # xóa gradient cũ
        
        outputs = model(inputs)  # Forward pass
        correct_train += calculate_accuracy(outputs,labels)
        labels = labels.unsqueeze(1).float()  # Ensure labels have the correct shape and type
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        
        running_loss += loss.item()
        if i % 2000 == 1999:  # Print every 2000 batches
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)
    train_accuracy = correct_train / len(train_loader)
    train_accuracies.append(train_accuracy)
    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            labels = labels.unsqueeze(1).float()
            correct_in = calculate_accuracy(outputs, labels)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            total += labels.size(0)
            correct += (correct_in)  # Accumulate accuracy
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    accuracy_percentage = (correct*100 / total) if total > 0 else 0
    val_accuracies.append(accuracy_percentage/100)
    print(f'Accuracy on validation dataset: {accuracy_percentage:.2f}%')

print('Finished Training')

Accuracy on validation dataset: 64.00%
Accuracy on validation dataset: 79.83%
Accuracy on validation dataset: 82.50%
Accuracy on validation dataset: 88.17%
Accuracy on validation dataset: 93.67%
Accuracy on validation dataset: 88.67%
Accuracy on validation dataset: 95.17%
Accuracy on validation dataset: 95.33%
Accuracy on validation dataset: 93.50%
Accuracy on validation dataset: 96.83%
Accuracy on validation dataset: 95.17%
Accuracy on validation dataset: 97.17%
Accuracy on validation dataset: 97.67%
Accuracy on validation dataset: 97.33%
Accuracy on validation dataset: 96.50%
Accuracy on validation dataset: 97.83%
Accuracy on validation dataset: 95.33%
Accuracy on validation dataset: 97.50%
Accuracy on validation dataset: 98.17%
Accuracy on validation dataset: 98.17%
Finished Training


In [1]:
#torch.save(model, 'BrainTumorbyTorch.pth')

In [14]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(range(1,21),train_losses)
plt.plot(range(1, 21), val_losses, label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()